## Fisher's Exact Test
[Lecture Notes](https://github.com/bcaffo/MathematicsBiostatisticsBootCamp2/blob/master/lecture7.pdf)

In Fisher's exact test, the number of treated and control cases, along with the total number of successes and failures must stay fixed. Under the null hypothesis, the probability of success is always the same regardless of whether the case is treated or not. By fixing the sums and studying all the permutations, the p-value is the probability of observing an equally or more extreme table than the observed table in favor of the alternative hypothesis.

This test is used to test the hypothesis that success probability in treatments is not equal (or larger or smaller) than that of the control. It is equivalent to a binomial test, however the binomial test requires large samples to ensure that the test statistic converges to a standard normal.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm, t, binom_test, beta
from math import comb

import itertools

In [2]:
# Lecture example
data = pd.DataFrame({
    'Tumor': [4, 2],
    'None': [1, 3]
}, index=['Treated', 'Control'])
data

,Tumor,None
Treated,4,1
Control,2,3


Testing the null hypothesis that the probability of a mouse having a tumor is equal for treated and control mice against the alternative hypothesis that treated mice are more likely to have a tumor.

In [3]:
def hypergeometric_prob(n1, n2, x, z):
    return comb(n1, x) * comb(n2, z-x) / comb(n1+n2, z)

We compute the probability of the observed table and the more extreme tables in favor of the alternative probability using the hypergeometric distribution. Our margins should stay unchanged when manipulating the table, i.e., the sum of each of tumor, non-tumor, control, and treated cases should not change

In [4]:
# Number of treated mice
n1 = 5
# Number of control mice
n2 = 5
# Total number of tumors
z = 6

In [5]:
# Observed table

# Number of tumors in treated mice
x = 4

p1 = hypergeometric_prob(n1, n2, x, z)
p1

0.23809523809523808

In [6]:
# More extreme table

# Number of tumors in treated mice
x = 5

p2 = hypergeometric_prob(n1, n2, x, z)
p2

0.023809523809523808

In [7]:
p_val = p1 + p2
p_val

0.26190476190476186

Using Scipy `fisher_exact`

In [8]:
from scipy.stats import fisher_exact

In [9]:
arr = data.to_numpy()

fisher_exact(arr, 'greater')

(6.0, 0.26190476190476203)

In [10]:
arr = data.to_numpy()

fisher_exact(arr, 'two-sided')

(6.0, 0.523809523809524)

Estimating Fisher's exact test p_value using a **Monte Carlo simulation**

In [11]:
# We have a total of 6 tumors and 10 study cases
# 1s represent tumors
# The first five are treated and the others are control
arr = np.array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

n = 10000
count = 0

for i in range(n):
    np.random.shuffle(arr)
    # When there are 4 or more tumors in the first five cases (treated cases)
    # we have an equally or more extreme case than the observed case
    if sum(arr[:5]) >= 4:
        count += 1
        
p_val = 1.*count / n
p_val

0.2618

## Chi-squared test
[Lecture Notes](https://github.com/bcaffo/MathematicsBiostatisticsBootCamp2/blob/master/lecture8.pdf)

* The Chi-Squared test is used also to test the significance of the difference between treatment and control (Tests independence of two “things”). 

* The Chi-squared test requires large cell counts

* The Chi-squared test statistic is computed as the sum of the squared differences between the observed count and the expected count divided by the expected count, where the expected count is calculated using the proportion of successes and failures under the null hypothesis.

* •	The alternative is always two sided

Simulating the couple's rating example using Scipy's `chi2_contingency` function and Monte Carlo simulation.

In [12]:
# Husband vs wife rating
obs_arr = np.array([
    [7, 7, 2, 3],
    [2, 8, 3, 7],
    [1, 5, 4, 9],
    [2, 8, 9, 14]
])

**1. Using Scipy's `chi2_contingency`:** [Docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html)

In [13]:
from scipy.stats import chi2_contingency, chi2, chisquare

In [14]:
obs_stat, p_val, dof, exp_vals = chi2_contingency(obs_arr)
print('Statistic: {:.3f}'.format(obs_stat))
print('P-value: {:.3f}'.format(p_val))

Statistic: 16.955
P-value: 0.049


**2. Monte Carlo simulation:**

Computing an exact p-value for the Chi-squared test.

In [15]:
# Total husband ratings
husband = obs_arr.sum(axis=1)

# Total wife ratings
wife = obs_arr.sum(axis=0)

In [16]:
n = 10000

husband_perm = np.array(list(''.join([num*sym for num, sym in zip(husband, 'NFVA')])))

wife_perm = np.array(list(''.join([num*sym for num, sym in zip(wife, 'NFVA')])))

# The following 4*4 array has the string-form permutation in each cell
all_combinations = list(map(lambda x: x[0] + x[1], itertools.product('NFVA', 'NFVA')))
combo_arr = np.array(all_combinations).reshape(4, 4)

mc_count = 0

for _ in range(n):
    table = np.zeros((4, 4))
    
    # Shuffling the husband array
    np.random.shuffle(husband_perm)
    
    # The combinations of this iteration
    combos = np.char.add(husband_perm, wife_perm)
    
    for combo in all_combinations:
        # Counting the number of each combination
        count = (combos == combo).sum()
        
        # Getting the index of the combination
        i, j = np.where(combo_arr == combo)
        
        # Assigniong the count to that index
        table[i, j] = count
    
    # Computing the chi-squared statistic
    iter_stat, p_val, dof, exp_vals = chi2_contingency(table)
    
    if iter_stat > obs_stat:
        mc_count += 1
        

exact_pval = 1. * mc_count / n

print('MC exact p-value: {:.3f}'.format(exact_pval))

MC exact p-value: 0.051


### Goodness of fit testing
In this case, we want to test our observations against a theoretical set of probabilities that we have.

**Testing the numpy random number generator**:

We define a set of probabilities that a random number generator should apply to

In [17]:
def test_random_gen(n, prob_range):
    
    num_arr = np.random.uniform(size=n)
    
    count_arr = np.zeros(prob_range.shape[0]-1)
    i = 0
    
    for i, (low_bound, upp_bound) in enumerate(zip(prob_range[:-1], prob_range[1:])):
        
        cond = (num_arr >= low_bound) & (num_arr < upp_bound)
        count = cond.sum()
        count_arr[i] = count

    
    exp_arr = np.diff(prob_range) * n
    
    return count_arr, exp_arr

In [18]:
n = 10000
prob_range = np.linspace(0, 1, 11)

count_arr, exp_arr = test_random_gen(n, prob_range)

# Applying chi2 test
stat = np.sum((count_arr - exp_arr)**2 / exp_arr)
df = count_arr.shape[0] - 1

p_val = 1 - chi2.cdf(x=stat, df=df)

print('My test:')
print('Statistic: {:.3f}'.format(stat))
print('P-value: {:.3f}'.format(p_val))

print('\n')

# Scipy chi-squared test
stat, p_val = chisquare(count_arr, exp_arr)

print('Scipy test:')
print('Statistic: {:.3f}'.format(stat))
print('P-value: {:.3f}'.format(p_val))

My test:
Statistic: 10.580
P-value: 0.306


Scipy test:
Statistic: 10.580
P-value: 0.306
